In [1]:
import os
import glob
import argparse 
import subprocess as sub
import shlex 
from subprocess import PIPE, Popen
import pandas as pd
import logging
import re
import datetime
import sys

In [46]:
def getting_bnum_tnum_list(bnum_tnum_csv): 
    ########### Get bnum_tnum list: 
    savedir = os.getcwd()
    os.chdir(csv_dir)
    bnum_tnum_df = pd.read_csv(bnum_tnum_csv, header = None)
    bnum_tnum_df.columns = ['bnum', 'tnum', 'DUMMY']
    os.chdir(savedir)
    return bnum_tnum_df

In [47]:
def get_diffu_data(measure = "median"):
    os.chdir(recgli_path_root+"/"+bnum+"/"+tnum)
    if len(glob.glob('svk_roi_analysis/*adcfa1000.csv'))>0:
        ## read in data
        diffu_data_file = pd.read_csv(glob.glob('svk_roi_analysis/*adcfa1000.csv')[0])
        if len(vial_files)>0: 
            ## shorten vial names
            vial_names = [vial.replace(".", "_").split("_")[2] for vial in vial_files]
            ## shorten the diffuomic data down to only the biopsies of interest: 
            diffu_data_biopsy_file = diffu_data_file[diffu_data_file['roi-label'].isin(vial_names)]
            ## find only the measure of interest: 
            diffu_data_biopsy_file_measure = diffu_data_biopsy_file[diffu_data_biopsy_file['measure']==measure]
        else: 
            vial_names = []
            print("ERROR! No biopsies.")
    else: 
        print('ERROR! No diffuomic data present.')
    return diffu_data_biopsy_file_measure

In [56]:
def get_ev_data(measure = "median"):
    os.chdir(recgli_path_root+"/"+bnum+"/"+tnum)
    if len(glob.glob('svk_roi_analysis/*ev1ev2ev31000.csv'))>0:
        ## read in data
        ev_data_file = pd.read_csv(glob.glob('svk_roi_analysis/*ev1ev2ev31000.csv')[0])
        if len(vial_files)>0: 
            ## shorten vial names
            vial_names = [vial.replace(".", "_").split("_")[2] for vial in vial_files]
            ## shorten the diffusion data down to only the biopsies of interest: 
            ev_data_biopsy_file = ev_data_file[ev_data_file['roi-label'].isin(vial_names)]
            ## find only the measure of interest: 
            ev_data_biopsy_file_measure = ev_data_biopsy_file[ev_data_biopsy_file['measure']==measure]
        else: 
            vial_names = []
            print("ERROR! No biopsies.")
    else: 
        print('ERROR! No EV data present.')
    return ev_data_biopsy_file_measure

In [57]:

#####################################
# EXAMPLE: get_diffu_biopsy_data.py --csv_name /home/sf6735452/DataWrangling/GetMergeData/Oct2018/ajnr_
#            --cohort_name REC_HGG
#            --output_file REC_HGG_diffu_biopsies.csv --output_dir ./
#####################################

parser = argparse.ArgumentParser(description='Create a systematic logger of whether biopsies are valid or not through visual inspection')
parser.add_argument("--csv_name",        required=True,    help='Precise name of the csv file that contains the perfusion files of interest.')
parser.add_argument("--csv_dir",         required=True,    help='Precise path of the csv file that contains the perfusion files of interest.')
parser.add_argument("--cohort_name",     required=True,    help='Precise cohort name of the scans of interest (e.g. "po1_preop_recur" or "REC_HGG")')
parser.add_argument("--output_file",     required=True,    help="Name of the output file csv")
parser.add_argument("--output_dir",      required=True,    help="Path where output files get written ")
parser.add_argument("--measure",         required=False,   help='Measurement that you desire for the diffuomical imaging (e.g. median, mean, max, etc.)', default = "median")
parser.add_argument("-v", "--verbose",                     help = "verbose output", action='store_true', default=False,   required=False)


_StoreTrueAction(option_strings=['-v', '--verbose'], dest='verbose', nargs=0, const=True, default=False, type=None, choices=None, help='verbose output', metavar=None)

In [58]:
args = parser.parse_args("--csv_dir /home/sf673542/MultiParametricMRI/MP_MRI_Final/GetMergeData/Get_REC_HGG_Data/ --csv_name rec_hgg_process_list.79.csv --cohort_name REC_HGG --output_file REC_HGG_diffu1000_biopsies.csv --output_dir /home/sf673542/MultiParametricMRI/MP_MRI_Final/GetMergeData/Get_REC_HGG_Data/ --measure median".split())


In [59]:
# args = parser.parse_args("--csv_dir /home/sf673542/MultiParametricMRI/MP_MRI_Final/GetMergeData/Get_Old_P01/ --csv_name po1_preop_recur_process_list.125.csv --cohort_name po1_preop_recur --output_file po1_preop_recur_diffu1000_biopsies.csv --output_dir /home/sf673542/MultiParametricMRI/MP_MRI_Final/GetMergeData/Get_Old_P01/ --measure median".split())


In [60]:

#####################################
#   Create strings of the arguments for 
#   navigating to correct directory
#####################################
cohort_name     = args.cohort_name
csv_name        = args.csv_name
csv_dir         = args.csv_dir
output_file     = args.output_file
output_dir      = args.output_dir
measure         = args.measure

print("===============================================")
print("scan list dir:     ", csv_dir)
print("scan list name:    ", csv_name)
print("cohort name:       ", cohort_name) 
print("output file name:  ", output_file) 
print("output dir:        ", output_dir)
print("===============================================")


scan list dir:      /home/sf673542/MultiParametricMRI/MP_MRI_Final/GetMergeData/Get_REC_HGG_Data/
scan list name:     rec_hgg_process_list.79.csv
cohort name:        REC_HGG
output file name:   REC_HGG_diffu1000_biopsies.csv
output dir:         /home/sf673542/MultiParametricMRI/MP_MRI_Final/GetMergeData/Get_REC_HGG_Data/


In [61]:

#####################################
#   Reading in the csv_name
#   as the scan listls
#####################################
bnum_tnum_df = getting_bnum_tnum_list(csv_name)

In [62]:


#####################################
#   Setting the roots of the data 
#   based on the cohort name
#####################################

if cohort_name == 'po1_preop_recur': 
    recgli_path_root = '/data/RECglioma/archived/'
elif cohort_name == 'REC_HGG': 
    recgli_path_root = '/data/RECglioma/'
else: 
    print('Please use a valid cohort name, REC_HGG or po1_preop_recur.')
    exit(1)


In [63]:

#####################################
#   Instantiate the total diffuomical dataframe
#####################################

diffu_total_df = pd.DataFrame()
ev_total_df  = pd.DataFrame()


In [64]:
error_log = pd.DataFrame()


In [65]:

#####################################
#   Iterating through scans, grabbing the data of interest: 
#####################################

for index, row in bnum_tnum_df.iterrows():
    bnum = row['bnum']
    tnum = row['tnum']
    error_log_line = {'bnum': bnum, 'tnum': tnum}
    
    print(bnum)
    print(tnum)

    #####################################
    #   Change into the correct directory
    #####################################
    os.chdir(recgli_path_root+"/"+bnum+"/"+tnum)

    #####################################
    #   Gather the vialIDs 
    #####################################
    if "roi_analysis" in os.listdir(): 
        os.chdir('roi_analysis')
        vial_files = glob.glob('*_t1ca_*.idf')
    else: 
        vial_files = []
        print('ERROR! No biopsies.')

    #####################################
    #   Change back into the correct directory
    #####################################
    os.chdir(recgli_path_root+"/"+bnum+"/"+tnum)

    #####################################
    #   Gather the data if available: 
    #####################################

    ## If there exists an diffuomical data file: 
    if len(glob.glob('svk_roi_analysis/*adcfa1000.csv'))>0: 
        print('diffusion csv file found')         
        ## If there exists biopsy masks in roi_analysis: 
        if len(vial_files)>0: 
            print('vialIDs found in roi_analysis')
            ## Set the vial_file_status = 1 so that we know they exist 
            vial_files_status = 1

            ## Instanstiate the tnum_diffuomical_df
            tnum_diffuomical_df  = pd.DataFrame()

            ## Get the data: 
            tnum_diffuomical_df = get_diffu_data()

            ## Add these data to the overall dataFrame: 
            diffu_total_df = diffu_total_df.append(tnum_diffuomical_df, ignore_index= True)\

        else: 
            print('No vial IDs found! No biopsies.')
            error_log_line['biopsies']='error' 
            
    else: 
        print("------------------------------------------")
        print('No diffu1000.csv file! run svk_roi_analysis')
        print('------------------------------------------')
        error_log_line['data_adcfa_file']='error'
        
    ## If there exists an ev data file: 
    if len(glob.glob('svk_roi_analysis/*ev31000.csv'))>0: 
        print('ev 1000 csv file found')         
        ## If there exists biopsy masks in roi_analysis: 
        if len(vial_files)>0: 
            print('vialIDs found in roi_analysis')
            ## Set the vial_file_status = 1 so that we know they exist 
            vial_files_status = 1

            ## Instanstiate the tnum_diffusion_df
            tnum_ev_df  = pd.DataFrame()

            ## Get the data: 
            tnum_ev_df = get_ev_data()

            ## Add these data to the overall dataFrame: 
            ev_total_df = ev_total_df.append(tnum_ev_df, ignore_index= True)\

        else: 
            print('No vial IDs found! No biopsies.')

    else: 
        print("------------------------------------------")
        print('ERROR! No ev1000.csv file! run svk_roi_analysis')
        print('------------------------------------------')
        error_log_line['data_ev1000_file']='error' 
        
    error_log = error_log.append(error_log_line, ignore_index = True)

b1338
t8773
diffusion csv file found
vialIDs found in roi_analysis
ev 1000 csv file found
vialIDs found in roi_analysis
b2947
t8775
diffusion csv file found
vialIDs found in roi_analysis
ev 1000 csv file found
vialIDs found in roi_analysis
b3472
t8783
diffusion csv file found
vialIDs found in roi_analysis
ev 1000 csv file found
vialIDs found in roi_analysis
b3499
t8869
------------------------------------------
No diffu1000.csv file! run svk_roi_analysis
------------------------------------------
------------------------------------------
ERROR! No ev1000.csv file! run svk_roi_analysis
------------------------------------------
b3125
t8878
diffusion csv file found
vialIDs found in roi_analysis
ev 1000 csv file found
vialIDs found in roi_analysis
b3526
t8943
------------------------------------------
No diffu1000.csv file! run svk_roi_analysis
------------------------------------------
------------------------------------------
ERROR! No ev1000.csv file! run svk_roi_analysis
-----------

b4205
t10756
diffusion csv file found
vialIDs found in roi_analysis
ev 1000 csv file found
vialIDs found in roi_analysis
b4208
t10763
------------------------------------------
No diffu1000.csv file! run svk_roi_analysis
------------------------------------------
------------------------------------------
ERROR! No ev1000.csv file! run svk_roi_analysis
------------------------------------------
b4210
t10768
------------------------------------------
No diffu1000.csv file! run svk_roi_analysis
------------------------------------------
------------------------------------------
ERROR! No ev1000.csv file! run svk_roi_analysis
------------------------------------------
b4211
t10770
diffusion csv file found
vialIDs found in roi_analysis
ev 1000 csv file found
vialIDs found in roi_analysis
b3776
t10777
diffusion csv file found
vialIDs found in roi_analysis
ev 1000 csv file found
vialIDs found in roi_analysis
b4193
t10790
diffusion csv file found
vialIDs found in roi_analysis
ev 1000 csv fil

In [66]:
error_log.head(30)

,bnum,tnum,data_adcfa_file,data_ev1000_file,biopsies
0,b1338,t8773,NaN,NaN,NaN
1,b2947,t8775,NaN,NaN,NaN
2,b3472,t8783,NaN,NaN,NaN
3,b3499,t8869,error,error,NaN
4,b3125,t8878,NaN,NaN,NaN
5,b3526,t8943,error,error,NaN
6,b3527,t8944,NaN,NaN,NaN
7,b3533,t8956,NaN,NaN,NaN
8,b1837,t8960,NaN,NaN,NaN
9,b3541,t8977,NaN,NaN,NaN


In [34]:
print(diffu_total_df.columns)
print(ev_total_df.columns)

Index([], dtype='object')
Index([], dtype='object')


In [35]:
#####################################
#   Rearrange the data: 
#####################################

cols = ['t-num', 'roi-label', 'adc', 'fa', 'nadc', 'nfa']
diffu_total_df = diffu_total_df[cols]
diffu_total_df.columns = ['tnum', 'roi.label', 'adc.1', 'fa.1', "nadc.1", 'nfa.1']

cols_ev = ['t-num', 'roi-label', 'nev1', 'nev2', 'nev3', 'nevrad']
ev_total_df=ev_total_df[cols_ev]
ev_total_df.columns = ['tnum', 'roi.label','nev1.1', 'nev2.1', 'nev3.1', 'nevrad.1']


KeyError: "['t-num' 'roi-label' 'adc' 'fa' 'nadc' 'nfa'] not in index"

In [36]:
diffu_total_df.shape


(0, 0)

In [37]:
ev_total_df.shape


(0, 0)

In [38]:
new_diff_df = pd.merge(left = diffu_total_df, right = ev_total_df, how = "outer", on = ['roi.label', 'tnum'])

KeyError: 'roi.label'

In [39]:
new_diff_df.columns

Index(['tnum', 'roi.label', 'adc.1', 'fa.1', 'nadc.1', 'nfa.1', 'nev1.1',
       'nev2.1', 'nev3.1', 'nevrad.1'],
      dtype='object')

In [40]:
new_diff_df.shape
new_diff_df.head()

,tnum,roi.label,adc.1,fa.1,nadc.1,nfa.1,nev1.1,nev2.1,nev3.1,nevrad.1
0,t5317,11B16,1517.5,130.0,1.87,0.46,1.41,1.68,1.76,1.61
1,t5317,12B16,1542.5,130.0,1.90,0.46,1.41,1.72,1.76,1.61
2,t5333,51B16,932.5,190.0,1.32,1.06,1.11,1.18,1.21,1.19
3,t5333,52B16,1072.5,270.0,1.52,1.50,1.34,1.37,1.21,1.27
4,t5333,53B16,932.5,190.0,1.32,1.06,1.11,1.21,1.21,1.19


In [41]:
out_file = output_dir+output_file

out_file

'/home/sf673542/MultiParametricMRI/MP_MRI_Final/GetMergeData/Get_Old_P01/po1_preop_recur_diffu1000_biopsies.csv'

In [42]:
new_diff_df.to_csv(out_file, index=False)

In [43]:
error_log.to_csv('/home/sf673542/MultiParametricMRI/MP_MRI_Final/GetMergeData/Get_REC_HGG_Data/REC_HGG_diffu1000_error_log.csv')

In [45]:
# error_log.to_csv('/home/sf673542/MultiParametricMRI/MP_MRI_Final/GetMergeData/Get_Old_P01/po1_preop_recur_diffu1000_error_log.csv')
